# Heroes Of Pymoli Data Analysis

## Observed Trends

## Initial import

In [502]:
import pandas as pd
import json
import glob
import os

In [503]:
os.remove('finalFile.json')
glob_data = []
for file in glob.glob('*.json'):
    with open(file) as json_file:
        data = json.load(json_file)

        i = 0
        while i < len(data):
            glob_data.append(data[i])
            i += 1

with open('finalFile.json', 'w') as f:
    json.dump(glob_data, f, indent=4)

In [504]:
df = pd.read_json('finalFile.json')
#df.head(3)

## Player Count

In [505]:
#Total Number of Players
df['Unique_ID'] = df['SN'] + df['Gender']
total_players = len(df['Unique_ID'].unique())
tp_df = pd.DataFrame()
tp_df.set_value(0,'Total Players',total_players)

,Total Players
0,619.0


## Purchasing Analysis (Total)

In [506]:
#Number of Unique Items
unique_items = len(df['Item Name'].unique())
#Average Purchase Price
avg_price = df['Price'].mean()
#Total Number of Purchases
total_purchases = len(df)
#Total Revenue
total_revenue = df['Price'].sum()
#Create Summary dataframe
summary_df = pd.DataFrame()
summary_df.set_value(0,'Number of Unique Items',unique_items)
summary_df.set_value(0,'Average Price',avg_price)
summary_df.set_value(0,'Number of Purchases',total_purchases)
summary_df.set_value(0,'Total Revenue',total_revenue)
summary_df["Number of Unique Items"] = summary_df["Number of Unique Items"].map( '{:.0f}'.format)
summary_df["Number of Purchases"] = summary_df["Number of Purchases"].map( '{:.0f}'.format)
summary_df["Average Price"] = summary_df["Average Price"].map( '${:.2f}'.format)
summary_df["Total Revenue"] = summary_df["Total Revenue"].map( '${:.2f}'.format)
summary_df


,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,180,$2.93,858,$2514.43


## Gender Demographics

In [507]:
df_SN_Gender_wdups = df[['SN','Gender']]
df_SN_Gender = df_SN_Gender_wdups.drop_duplicates()
#df_SN_Gender.head()
#Count of Player Demographics
gender_breakdown = df_SN_Gender["Gender"].value_counts()#.to_frame().reset_index()
#Percentage of Player Demographics and convert to DataFrame
gender_breakdown_percent = (gender_breakdown / total_players * 100).to_frame().reset_index()
#Convert to DataFrame
gender_breakdown_df = gender_breakdown.to_frame().reset_index()
#Rename columns
gender_breakdown_percent.columns = ['Gender', 'Percentage of Players']
gender_breakdown_df.columns = ['Gender', 'Total Count']

In [508]:
#Merge tables create summary Demographics table and format
merge_table = pd.merge(gender_breakdown_percent, gender_breakdown_df, on='Gender')
merge_table["Percentage of Players"] = merge_table["Percentage of Players"].map( '{:.2f}%'.format)


## Purchasing Analysis (Gender)

In [509]:
#* The below each broken by gender
#  * Purchase Count
#  * Average Purchase Price
#  * Total Purchase Value
#  * Normalized Totals

df2 = df.groupby('Gender')
gender_count = df2['Age'].count().to_frame().reset_index()
gender_count.columns = ['Gender','Purchase Count']
gender_avg = df2['Price'].mean().to_frame().reset_index()
gender_avg.columns = ['Gender','Average Purchase Price']
gender_total_value = df2['Price'].sum().to_frame().reset_index()
gender_total_value.columns = ['Gender','Total Purchase Value']
gender_normalize = (df2['Price'].sum() / df_SN_Gender['Gender'].value_counts()).to_frame().reset_index()
gender_normalize.columns = ['Gender', 'Normalized Totals']
tmp1 = pd.merge(gender_count,gender_avg, on='Gender')
tmp2 = pd.merge(tmp1,gender_total_value, on='Gender')
purchase_analysis = pd.merge(tmp2,gender_normalize, on='Gender')
df_SN_Gender['Gender'].value_counts()

Male                     498
Female                   112
Other / Non-Disclosed      9
Name: Gender, dtype: int64

In [510]:
#Formatting
purchase_analysis["Average Purchase Price"] = purchase_analysis["Average Purchase Price"].map("${:.2f}".format)
purchase_analysis["Total Purchase Value"] = purchase_analysis["Total Purchase Value"].map("${:.2f}".format)
purchase_analysis["Normalized Totals"] = purchase_analysis["Normalized Totals"].map("${:.2f}".format)
purchase_analysis


,Gender,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
0,Female,149,$2.85,$424.29,$3.79
1,Male,697,$2.94,$2052.28,$4.12
2,Other / Non-Disclosed,12,$3.15,$37.86,$4.21


## Age Demographics 

In [511]:
#* The below each broken into bins of 4 years (i.e. &lt;10, 10-14, 15-19, etc.) 
#  * Purchase Count
#  * Average Purchase Price
#  * Total Purchase Value
#  * Normalized Totals
dfAge = df
bins = [0,9,14,19,24,29,34,39,100]
group_names = ['<10','10-14','15-19','20-24','25-29','30-34','35-39','40+']

In [512]:
dfAge['AgeGroup'] = pd.cut(df["Age"], bins, labels=group_names)
dfAge.head()
# with pd.option_context('display.max_rows', None, 'display.max_columns', 3):
#     display(dfAge.sort_values('Age'))

,Age,Gender,Item ID,Item Name,Price,SN,Unique_ID,AgeGroup
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34,Aelalis34Male,35-39
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46,Eolo46Male,20-24
2,34,Male,174,Primitive Blade,2.46,Assastnya25,Assastnya25Male,30-34
3,21,Male,92,Final Critic,1.36,Pheusrical25,Pheusrical25Male,20-24
4,23,Male,63,Stormfury Mace,1.27,Aela59,Aela59Male,20-24


In [513]:
df3 = df.groupby('AgeGroup')
age_count = df3['Age'].count().to_frame().reset_index()
age_count.columns = ['Age Group','Purchase Count']
age_avg = df3['Price'].mean().to_frame().reset_index()
age_avg.columns = ['Age Group','Average Purchase Price']
age_total_value = df3['Price'].sum().to_frame().reset_index()
age_total_value.columns = ['Age Group','Total Purchase Value']
age_normalize = (age_total_value['Total Purchase Value']/age_count['Purchase Count']).to_frame().reset_index(drop=True)
age_normalize['Age Group'] = group_names
cols = ['Normalized Totals','Age Group']
age_normalize.columns = cols
age_normalize = age_normalize[['Age Group','Normalized Totals']]
tmp_age1 = pd.merge(age_count,age_avg, on='Age Group')
tmp_age2 = pd.merge(tmp_age1,age_total_value, on='Age Group')
age_purchase_analysis = pd.merge(tmp_age2,age_normalize, on='Age Group')

In [514]:
age_purchase_analysis["Average Purchase Price"] = age_purchase_analysis["Average Purchase Price"].map("${:.2f}".format)
age_purchase_analysis["Total Purchase Value"] = age_purchase_analysis["Total Purchase Value"].map("${:.2f}".format)
age_purchase_analysis["Normalized Totals"] = age_purchase_analysis["Normalized Totals"].map("${:.2f}".format)
age_purchase_analysis

,Age Group,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
0,<10,33,$2.95,$97.28,$2.95
1,10-14,38,$2.79,$105.91,$2.79
2,15-19,144,$2.89,$416.83,$2.89
3,20-24,372,$2.92,$1087.66,$2.92
4,25-29,134,$2.96,$396.44,$2.96
5,30-34,71,$2.97,$211.14,$2.97
6,35-39,48,$2.93,$140.77,$2.93
7,40+,18,$3.24,$58.40,$3.24


## Top Spenders

In [515]:
# * Identify the the top 5 spenders in the game by total purchase value, then list (in a table):
top_spenders_group = df.groupby(['Unique_ID'])
tpv = top_spenders_group['Price'].sum().sort_values(ascending=False).head(5).to_frame().reset_index()
tpv.columns = ('Unique_ID','Total Purchase Value')
df_filtered = df[df['Unique_ID'].isin(tpv['Unique_ID'])]
df_filtered_group= df_filtered.groupby(['Unique_ID'])
pc = df_filtered_group['Age'].count().to_frame().reset_index()
pc.columns = ('Unique_ID', 'Purchase Count')
ts_merger = pd.merge(tpv,pc,on='Unique_ID')
ts_merger['Average Purchase Price'] = ts_merger["Total Purchase Value"] / ts_merger['Purchase Count'] 
ts_merger['Average Purchase Price'] = ts_merger['Average Purchase Price'].map("${:.2f}".format)
ts_merger["Purchase Count"] = ts_merger["Purchase Count"].map("{:.0f}".format)
ts_merger["Total Purchase Value"] = ts_merger["Total Purchase Value"].map("${:.2f}".format)
ts_merger = ts_merger[['Unique_ID',"Purchase Count",'Average Purchase Price',"Total Purchase Value"]]
ts_merger

,Unique_ID,Purchase Count,Average Purchase Price,Total Purchase Value
0,Undirrala66Male,5,$3.41,$17.06
1,Aerithllora36Male,4,$3.77,$15.10
2,Saedue76Male,4,$3.39,$13.56
3,Sondim43Male,4,$3.25,$13.02
4,Mindimnya67Female,4,$3.18,$12.74


# Most Popular Items

In [516]:
top_items_group = df.groupby(['Item ID'])
ti_df = top_items_group['Price'].count().sort_values(ascending=False).head(5).to_frame().reset_index()
ti_df.columns = ('Item ID', 'Purchase Count')
ti_df_filtered = df[df['Item ID'].isin(ti_df['Item ID'])]
ti_df_group = ti_df_filtered.groupby(['Item ID'])
tip = ti_df_group['Price'].sum().to_frame().reset_index()
tip.columns = ('Item ID', 'Total Purchase Value')
itemname = ti_df_filtered[['Item ID','Item Name']].drop_duplicates()
item_merger = pd.merge(ti_df, tip, on= 'Item ID')
item_merger2 = pd.merge(item_merger,itemname, on='Item ID') 
item_merger2['Average Item Price'] = item_merger2["Total Purchase Value"] / item_merger2['Purchase Count'] 
pop_items = item_merger2[['Item ID','Item Name','Purchase Count','Average Item Price','Total Purchase Value']]
pop_items['Average Item Price'] = pop_items['Average Item Price'].map("${:.2f}".format)
pop_items['Purchase Count'] = pop_items['Purchase Count'].map("{:.0f}".format)
pop_items['Total Purchase Value'] = pop_items['Total Purchase Value'].map("${:.2f}".format)
pop_items

,Item ID,Item Name,Purchase Count,Average Item Price,Total Purchase Value
0,84,Arcane Gem,12,$2.45,$29.34
1,39,"Betrayal, Whisper of Grieving Widows",11,$2.35,$25.85
2,31,Trickster,10,$2.32,$23.22
3,34,Retribution Axe,9,$4.14,$37.26
4,44,Bonecarvin Battle Axe,9,$2.67,$24.04


# Most Profitable Items

In [517]:
top_items_group = df.groupby(['Item ID'])
ti_df = top_items_group['Price'].sum().sort_values(ascending=False).head(5).to_frame().reset_index()
ti_df.columns = ('Item ID', 'Total Purchase Value')
ti_df_filtered = df[df['Item ID'].isin(ti_df['Item ID'])]
ti_df_group = ti_df_filtered.groupby(['Item ID'])
ti_count = ti_df_group['Price'].count().to_frame().reset_index()
ti_count.columns = ('Item ID', 'Purchase Count')
itemname = ti_df_filtered[['Item ID','Item Name']].drop_duplicates()
item_merger = pd.merge(ti_df, ti_count, on= 'Item ID')
item_merger2 = pd.merge(item_merger,itemname, on='Item ID') 
item_merger2['Average Item Price'] = item_merger2["Total Purchase Value"] / item_merger2['Purchase Count'] 
prof_items = item_merger2[['Item ID','Item Name','Purchase Count','Average Item Price','Total Purchase Value']]
prof_items['Average Item Price'] = prof_items['Average Item Price'].map("${:.2f}".format)
prof_items['Purchase Count'] = prof_items['Purchase Count'].map("{:.0f}".format)
prof_items['Total Purchase Value'] = prof_items['Total Purchase Value'].map("${:.2f}".format)
prof_items

,Item ID,Item Name,Purchase Count,Average Item Price,Total Purchase Value
0,34,Retribution Axe,9,$4.14,$37.26
1,107,"Splitter, Foe Of Subtlety",9,$3.67,$33.03
2,115,Spectral Diamond Doomblade,7,$4.25,$29.75
3,32,Orenmir,6,$4.95,$29.70
4,84,Arcane Gem,12,$2.45,$29.34
